In [54]:
# Importing the required libraries
import os
import cv2
import glob
import zipfile
import numpy as np
from tqdm import tqdm
from urllib import request
from google.colab import drive

In [55]:
#you need to replace this by the location of the dataset on your drive
# datasetloc="/content/drive/MyDrive/Hemangini Projects/dataset"
#! wget -q https://storage.googleapis.com/drive-bulk-export-anonymous/20220428T122708Z/4133399871716478688/aa76fc5c-9354-4b5f-ae56-aca2048e1129/1/acf3a9e4-29f2-415b-86b6-f7ba375e19ee?authuser -O dataset.zip
# drive.mount('/content/drive')
# datasetloc="/content/drive/MyDrive/Colab Notebooks/dataset"

# ! wget -q https://tinyurl.com/r5ut6tjx -O dataset.zip
# ! unzip -oqq dataset
# datasetloc="./dataset"


#if not os.path.exists("./dataset.zip"):
#    response = request.urlretrieve("https://drive.google.com/file/d/1z_B2zhcjl0heRbdRGUTQKwC82WqDjBLb/view?usp=sharing", "dataset.zip")
#    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
#        zip_ref.extractall(".")
#datasetloc="./dataset"

drive.mount('/content/drive')
datasetloc = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/dataset"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
# Return a list of path for the image in the folder
fakefiles=glob.glob(datasetloc+"/training_fake/*.jpg")

In [57]:
#reading the fake images and labelling them as 1
#tqdm helps to keep track of progress
data=[]
y=[]
for f in tqdm(fakefiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  
  data.append(img)
  y.append(1)

100%|██████████| 970/970 [00:19<00:00, 50.25it/s]


In [58]:
realfiles=glob.glob(datasetloc+"/training_real/*.jpg")

In [59]:
#reading the real images and labelling them as 0
# resize image so we can stay consistent with the uniformity throughout
for f in tqdm(realfiles):
  # twodimg= np.zeros((128,128))
  img=cv2.imread(f)
  img = cv2.resize(img  , (128 , 128))
  
  data.append(img)
  y.append(0)
  

100%|██████████| 1081/1081 [00:19<00:00, 55.95it/s]


In [60]:
#convering lists into arrays // Arrays if all arrays
data = np.asarray(data)
y = np.asarray(y)

In [61]:
#then we do the test train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)

In [62]:
#importing the libraries required for our cnn model
from keras.models import Sequential 
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop

In [63]:
model = Sequential()

model.add(Conv2D(32,(3,3),input_shape = (128,128,3),activation = 'relu',padding='same',))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [64]:
#fitting our model on the dataset
model.fit(X_train,y_train,epochs=10,batch_size=40)

Epoch 1/10
41/41 [==============================] - 62s 1s/step - loss: 45.5067 - accuracy: 0.4957
Epoch 2/10
41/41 [==============================] - 54s 1s/step - loss: 0.6923 - accuracy: 0.5073
Epoch 3/10
41/41 [==============================] - 57s 1s/step - loss: 0.6928 - accuracy: 0.5341
Epoch 4/10
41/41 [==============================] - 53s 1s/step - loss: 0.6908 - accuracy: 0.5268
Epoch 5/10
41/41 [==============================] - 56s 1s/step - loss: 0.6907 - accuracy: 0.5329
Epoch 6/10
41/41 [==============================] - 54s 1s/step - loss: 0.6854 - accuracy: 0.5506
Epoch 7/10
41/41 [==============================] - 55s 1s/step - loss: 0.6781 - accuracy: 0.5457
Epoch 8/10
41/41 [==============================] - 55s 1s/step - loss: 0.6734 - accuracy: 0.5524
Epoch 9/10
41/41 [==============================] - 53s 1s/step - loss: 0.6813 - accuracy: 0.5561
Epoch 10/10
41/41 [==============================] - 52s 1s/step - loss: 0.6629 - accuracy: 0.5683


In [65]:
ypredtrain=model.predict(X_train)
ypredtest=model.predict(X_test)

In [66]:
yptrain=[]
yptest=[]
for y in ypredtrain:
  if(y>=0.5):
    yptrain.append(1)
  else:
    yptrain.append(0)
for y in ypredtest:
  if(y>=0.5):
    yptest.append(1)
  else:
    yptest.append(0)


In [67]:
from sklearn.metrics import accuracy_score
print("the accuracy score for training set is",accuracy_score(y_train, yptrain))
print("the accuracy score for test set is",accuracy_score(y_test, yptest))

the accuracy score for training set is 0.5841463414634146
the accuracy score for test set is 0.5474452554744526
